In [ ]:
### please specify your input path here
PROJECT_FOLDER = "YOUR_PROJECT_FOLDER" # parent folder of the input images
IMAGE_DATA_FOLDER = PROJECT_FOLDER + "images/" # folder of the input images
INPUT_TEST_CSV_FILE = "YOUR_TEST_FILE" # csv file list locations / paths to test cases (dicom)
OUTPUT_FILE = "YOUR_OUTPUT" # in csv format

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import json
import sys
import importlib
import multiprocessing as mp
import gc
from tqdm.auto import tqdm
import glob
import torch
from copy import copy
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader
import timm
import pylibjpeg
import pydicom
from PIL import Image
import re
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gc.collect()

In [5]:
pd.__version__ # please use 1.3.5. One operation is not supported in later versions.

'1.3.5'

In [6]:
torch.backends.cudnn.benchmark = True

sys.path.append('./configs')
sys.path.append('./data')
sys.path.append('./models')
sys.path.append('./postprocess')
sys.path.append('./github-rsna2022')

In [7]:
COMP_FOLDER = PROJECT_FOLDER
DATA_FOLDER = IMAGE_DATA_FOLDER
META_DF = INPUT_TEST_CSV_FILE

test_df = pd.read_csv(INPUT_TEST_CSV_FILE)#.iloc[:1]

keycols = 'StudyInstanceUID slice_number'.split()

entries = []
for key, row in test_df.iterrows():
    image_index = [int(re.findall(r'(\d+).d', i)[0]) for i in glob.glob(row['image_folder']+'/*')]
    for im_i in image_index:
        entries.append([row['StudyInstanceUID'], row['image_folder'], im_i])
    # break # for debug use!
    

df = pd.DataFrame(entries, columns = ['StudyInstanceUID', 'image_folder', 'slice_number'])
df['slice_number'] = df['slice_number'].astype(int)
df = df.sort_values(keycols).reset_index(drop = True)

N_CORES = mp.cpu_count()
MIXED_PRECISION = False
PIN_MEMORY = True
DL_PREFETCH_FACTOR = 1
NUM_SLICES_PER_UID_BBOX = 600

FOLD = 3
MAX_FOLDS = 99

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

df['StudyInstanceUID'] = df.StudyInstanceUID.astype('category')
df['fold'] = -1

print(test_df.shape)
print(df.shape)
gc.collect()

(5161, 3)
(1662687, 4)


0

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# change it to nn.BCELoss(reduction='none') if you have sigmoid activation in last layer
loss_fn = torch.nn.BCELoss(reduction="none") 
competition_weights = {
    '-' : torch.tensor([7, 1, 1, 1, 1, 1, 1, 1], dtype=torch.float, device=device),
    '+' : torch.tensor([14, 2, 2, 2, 2, 2, 2, 2], dtype=torch.float, device=device),
}

In [9]:
def get_cfg(CFG):
    cfg = importlib.import_module('default_config')
    importlib.reload(cfg)
    cfg = importlib.import_module(CFG)
    importlib.reload(cfg)
    cfg = copy(cfg.cfg)
    cfg.post_process_pipeline = importlib.import_module(cfg.post_process_pipeline).post_process_pipeline

    cfg.data_dir = COMP_FOLDER
    cfg.test_data_folder = DATA_FOLDER
    cfg.mixed_precision = MIXED_PRECISION
    cfg.pretrained = False
    cfg.pretrained_weights = False
    cfg.batch_size = cfg.batch_size
    cfg.offline_inference = True

    print(CFG, cfg.model, cfg.dataset, cfg.backbone, cfg.pretrained_weights, cfg.post_process_pipeline)
    
    return cfg

def get_dl(cfg):
    ds = importlib.import_module(cfg.dataset)
    importlib.reload(ds)

    CustomDataset = ds.CustomDataset
    batch_to_device = ds.batch_to_device

    test_ds = CustomDataset(df, cfg, cfg.val_aug, mode="test")
    test_dl = DataLoader(test_ds, shuffle=False, batch_size=cfg.batch_size, collate_fn=ds.val_collate_fn, num_workers=N_CORES, pin_memory=PIN_MEMORY, prefetch_factor = DL_PREFETCH_FACTOR)

    return test_dl, batch_to_device

def get_bb_dl(cfg, bbdf):
    ds = importlib.import_module(cfg.dataset)
    importlib.reload(ds)

    CustomDataset = ds.CustomDataset
    batch_to_device = ds.batch_to_device

    test_ds = CustomDataset(bbdf, cfg, cfg.val_aug, mode="test")
    test_dl = DataLoader(test_ds, shuffle=False, batch_size=cfg.batch_size, collate_fn=ds.val_collate_fn, num_workers=N_CORES, pin_memory=PIN_MEMORY, prefetch_factor = DL_PREFETCH_FACTOR)

    return test_dl, batch_to_device

def get_state_dict(sd_fp):
    sd = torch.load(sd_fp, map_location="cpu")
    sd = {k.replace("module.", ""):v for k,v in sd.items()}
    return sd

def get_nets(cfg,state_dicts,test_ds):
    model = importlib.import_module(cfg.model)
    importlib.reload(model)
    Net = model.Net

    nets = []

    for i,state_dict in enumerate(state_dicts):
        net = Net(cfg).eval().to(DEVICE)
        print("loading dict")
        sd = get_state_dict(state_dict)
        if "model" in sd.keys():
            sd = sd["model"]
        net.load_state_dict(sd, strict=True)
        nets += [net.half()]
        del sd
        gc.collect()
    return nets

def competiton_loss_row_norm(y_hat, y):
    loss = loss_fn(y_hat, y)
    weights = y * competition_weights['+'] + (1 - y) * competition_weights['-']
    loss = (loss * weights).sum(axis=1)
    w_sum = weights.sum(axis=1)
    loss = torch.div(loss, w_sum)
    return loss.mean().item()

def window_range(g, thresh = 0.1, window = 5, min_periods=3, center=True):
    bbseqprobas = g.has_bbox.rolling(window, min_periods=min_periods, center=center).mean()
    if bbseqprobas.max() <= thresh:
        sn_from, sn_to = g.slice_numbers.iloc[[0,-1]]
        return [sn_from, sn_to]
    bb_range = np.where(bbseqprobas > thresh)[0]
    sn_from = g.slice_numbers.iloc[bb_range[0]]
    sn_to = g.slice_numbers.iloc[bb_range[-1]]
    return [sn_from, sn_to]

In [10]:
bbdf = df.copy()

In [ ]:
name = 'cfg_loc_dh_01B'

cfg = get_cfg(name)
cfg.meta_df = META_DF
cfg.data_folder = DATA_FOLDER
cfg.verify_sample = False
cfg.load_jpg = False
cfg.batch_size = 8
cfg.drop_scans = []
test_dl, batch_to_device = get_bb_dl(cfg, bbdf)
state_dict_fps = sorted(glob.glob('./weights-cfg-loc-dh-01b-all-fold-1/fold*check*'))[:MAX_FOLDS]
print('\n'.join(state_dict_fps))
nets = get_nets(cfg,state_dict_fps, test_dl.dataset)

In [ ]:
val_data = {}
preds = []
uids = []
slices = []
#ids = []
with torch.inference_mode():
    for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
        batch = batch_to_device(batch,DEVICE)
        batch['image'] = batch['image'].half()
        outs = [net(batch) for net in nets]
        preds += [torch.stack([out['preds'] for out in outs], dim=0).cpu().mean(0)]
        uids +=  [outs[0]['StudyUID'].cpu()]
        slices += [outs[0]['slice_numbers'].cpu()]
        if tt % 5 == 0:
            gc.collect()
            torch.cuda.empty_cache()
            
        #ids += batch['StudyUID'].cpu()
preds = torch.cat(preds, dim=0)
uids = torch.cat(uids, dim=0)
slices = torch.cat(slices, dim=0)


 27%|██▋       | 18484/69067 [37:10<1:09:45, 12.09it/s] 

In [ ]:
predbbdf = pd.DataFrame(preds.float().clip(0, 1).numpy(), columns = 'x0 y0 x1 y1 has_bbox'.split())
predbbdf['uid'] = uids.numpy()
predbbdf['slice_numbers'] = slices.numpy()[:,1]
predbbdf['slice_numbers_all'] = slices.numpy().tolist()
predbbdf['StudyInstanceUID'] = test_dl.dataset.df.loc[test_dl.dataset.ids]['StudyInstanceUID'].values
predbbdf['StudyInstanceUID'] = predbbdf['StudyInstanceUID'].astype('category')

In [ ]:
allpreddf = predbbdf
allpreddf = allpreddf.explode('slice_numbers_all')
allpreddf['slice_numbers'] = allpreddf['slice_numbers_all']
allpreddf = allpreddf.set_index('StudyInstanceUID').drop(['slice_numbers_all', 'uid'], axis=1)

allpreddf.to_csv('train_bbox_all_pred_v02.csv')

# Hack for studies with no bounding box > 0.5.... - just put in a dummy box and let it go thru the pipeline
predbbdf['max_has_bbox'] = predbbdf.groupby('StudyInstanceUID')['has_bbox'].transform(max)
predbbdf.loc[predbbdf.max_has_bbox<0.5, 'x0 y0'.split()] = 0.2
predbbdf.loc[predbbdf.max_has_bbox<0.5, 'x1 y1'.split()] = 0.8
predbbdf.loc[predbbdf.max_has_bbox<0.5, 'has_bbox'.split()] = 0.51
predbbdf = predbbdf.drop('max_has_bbox', axis=1)

bbpreddf = pd.concat([ \
    predbbdf.query('has_bbox > 0.5').groupby('StudyInstanceUID')['x0 y0'.split()].apply(min),
    predbbdf.query('has_bbox > 0.5').groupby('StudyInstanceUID')['x1 y1'.split()].apply(max)], axis=1)

wrls = predbbdf.groupby('StudyInstanceUID').apply(window_range)
wrdf = pd.DataFrame(wrls.tolist(), index = wrls.index, columns = 'slnum_from slnum_to'.split())
bbpreddf['slnum_from slnum_to'.split()] = wrdf.loc[bbpreddf.index]
bbpreddf['slnum_max'] = predbbdf.groupby('StudyInstanceUID')['slice_numbers'].max().loc[bbpreddf.index]

bbpreddf.iloc[:,:2] = np.floor((bbpreddf.iloc[:,:2] * 512)).astype(int).clip(0, 512)
bbpreddf.iloc[:,2:4] = np.ceil((bbpreddf.iloc[:,2:4] * 512)).astype(int).clip(0, 512)
bbpreddf.to_csv('train_bbox_pred_v01.csv')

((bbpreddf.slnum_to - bbpreddf.slnum_from) / bbpreddf.slnum_max).hist(bins = 50 )

In [ ]:
del test_dl, nets, preds, uids, slices, predbbdf, bbpreddf, allpreddf
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!head train_bbox_pred_v01.csv

In [ ]:
!head train_bbox_all_pred_v02.csv

In [ ]:
name = 'cfg_dh_fracseq_04F_crop3_gx1'

cfg = get_cfg(name)
cfg.meta_df = META_DF
cfg.data_folder = DATA_FOLDER
cfg.verify_sample = False
cfg.load_jpg = False
cfg.cnn_chunk_size = 32
cfg.bbox_df = 'train_bbox_pred_v01.csv'  #. f'{cfg.data_dir}/train_bbox_pred_v02.csv.gz'
cfg.bboxall_df = 'train_bbox_all_pred_v02.csv' # f'{cfg.data_dir}/train_bbox_all_pred_v02.csv.gz'
# cfg.bbox_df = 'train_bbox_pred_v01.csv'
# cfg.bbox_df = 'train_bbox_all_pred_v02.csv'

cfg.batch_size=1
#cfg.dataset = "ds_dh_fracseg_3F_crop_infer"
cfg.norm_on_cuda=True
test_dl, batch_to_device = get_dl(cfg)
test_dl.dataset.norm_mean = test_dl.dataset.norm_mean.to(DEVICE)#.half()
test_dl.dataset.norm_std = test_dl.dataset.norm_std.to(DEVICE)#.half()
#test_dl.dataset.df[cfg.target_seg] = 0.
#test_dl.dataset.df[cfg.target_frac] = 0.
test_dl.dataset.metadf = pd.DataFrame({'StudyInstanceUID': \
                                       test_dl.dataset.df.index.unique()}).set_index('StudyInstanceUID')
test_dl.dataset.metadf[['patient_overall'] + cfg.target] = 0.
#test_dl.dataset.metadf

state_dict_fps = sorted(glob.glob('./weights-cfg-dh-fracseq-04f-crop3-gx1-4f3ff/fold-1*check*'))
state_dict_fps = state_dict_fps[:MAX_FOLDS]

print('\n'.join(state_dict_fps))
nets = get_nets(cfg,state_dict_fps, test_dl.dataset)

In [ ]:
vaacoshl_data = {}
preds = []
#ids = []
with torch.inference_mode():
    for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
        batch = batch_to_device(batch,DEVICE)
        batch['image'] = test_dl.dataset.norm4d( batch['image'] )
        #batch['image'] = batch['image'].half()
        outs = [net(batch) for net in nets]
        preds += [torch.sigmoid(torch.stack([out['logits'] for out in outs], dim=0)).cpu().mean(0)]
        if tt % 10 == 0:
            gc.collect()
            torch.cuda.empty_cache()
ids = test_dl.dataset.ids
preds = torch.cat(preds, dim=0).float()
del test_dl, nets
gc.collect()
torch.cuda.empty_cache()

In [ ]:
preddf = pd.DataFrame(preds.numpy(), columns = ['patient_overall'] + cfg.target)
preddf['StudyInstanceUID'] = ids
preddf = preddf.set_index('StudyInstanceUID')
preddf.head()

In [ ]:
preddfls = [preddf]

In [ ]:
name = 'cfg_dh_fracseq_04F_crop_gx1'

cfg = get_cfg(name)
cfg.meta_df = META_DF
cfg.data_folder = DATA_FOLDER
cfg.verify_sample = False
cfg.load_jpg = False
cfg.cnn_chunk_size = 32
cfg.bbox_df = 'train_bbox_pred_v01.csv'
cfg.batch_size=1
cfg.norm_on_cuda=True
test_dl, batch_to_device = get_dl(cfg)
test_dl.dataset.norm_mean = test_dl.dataset.norm_mean.to(DEVICE)
test_dl.dataset.norm_std = test_dl.dataset.norm_std.to(DEVICE)
test_dl.dataset.metadf = pd.DataFrame({'StudyInstanceUID': \
                                       test_dl.dataset.df.index.unique()}).set_index('StudyInstanceUID')
test_dl.dataset.metadf[['patient_overall'] + cfg.target] = 0.

state_dict_fps = sorted(glob.glob('./weights-cfg-dh-fracseq-04f-crop-gx1-6ff/fold-1*check*'))
state_dict_fps = state_dict_fps[:MAX_FOLDS]

print('\n'.join(state_dict_fps))
nets = get_nets(cfg,state_dict_fps, test_dl.dataset)

In [ ]:
name = 'cfg_dh_fracseq_04F_crop_gx2'

cfg = get_cfg(name)
cfg.meta_df = META_DF
cfg.data_folder = DATA_FOLDER
cfg.verify_sample = False
cfg.load_jpg = False
cfg.cnn_chunk_size = 32
cfg.bbox_df = 'train_bbox_pred_v01.csv'
cfg.batch_size=1
#cfg.dataset = "ds_dh_fracseg_3F_crop_infer"
cfg.norm_on_cuda=True
test_dl, batch_to_device = get_dl(cfg)
test_dl.dataset.norm_mean = test_dl.dataset.norm_mean.to(DEVICE)#.half()
test_dl.dataset.norm_std = test_dl.dataset.norm_std.to(DEVICE)#.half()
#test_dl.dataset.df[cfg.target_seg] = 0.
#test_dl.dataset.df[cfg.target_frac] = 0.
test_dl.dataset.metadf = pd.DataFrame({'StudyInstanceUID': \
                                       test_dl.dataset.df.index.unique()}).set_index('StudyInstanceUID')
test_dl.dataset.metadf[['patient_overall'] + cfg.target] = 0.
#test_dl.dataset.metadf

state_dict_fps = sorted(glob.glob('./weights-cfg-dh-fracseq-04f-crop-gx2-4f2ff/fold-1*check*'))
state_dict_fps = state_dict_fps[:MAX_FOLDS]

print('\n'.join(state_dict_fps))
nets += get_nets(cfg,state_dict_fps, test_dl.dataset)

In [ ]:
name = 'cfg_dh_fracseq_04G_crop_gx1'

cfg = get_cfg(name)
cfg.meta_df = META_DF
cfg.data_folder = DATA_FOLDER
cfg.verify_sample = False
cfg.load_jpg = False
cfg.cnn_chunk_size = 16
cfg.bbox_df = 'train_bbox_pred_v01.csv'
cfg.batch_size=1
cfg.norm_on_cuda=True
test_dl, batch_to_device = get_dl(cfg)
test_dl.dataset.norm_mean = test_dl.dataset.norm_mean.to(DEVICE)#.half()
test_dl.dataset.norm_std = test_dl.dataset.norm_std.to(DEVICE)#.half()
test_dl.dataset.metadf = pd.DataFrame({'StudyInstanceUID': \
                                       test_dl.dataset.df.index.unique()}).set_index('StudyInstanceUID')
test_dl.dataset.metadf[['patient_overall'] + cfg.target] = 0.
state_dict_fps = sorted(glob.glob('./weights-cfg-dh-fracseq-04g-crop-gx1-5f2ff/fold-1*check*'))
state_dict_fps = state_dict_fps[:MAX_FOLDS]

print('\n'.join(state_dict_fps))
nets += get_nets(cfg,state_dict_fps, test_dl.dataset)

In [ ]:
print(len(nets))
print('Cnn chunk sizes : '+' '.join(map(str, [i.cfg.cnn_chunk_size for i in nets])))

In [ ]:
vaacoshl_data = {}
preds = []
#ids = []
with torch.inference_mode():
    for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
        batch = batch_to_device(batch,DEVICE)
        batch['image'] = test_dl.dataset.norm4d( batch['image'] )
        #batch['image'] = batch['image'].half()
        outs = [net(batch) for net in nets]
        preds += [torch.sigmoid(torch.stack([out['logits'] for out in outs], dim=0)).cpu().mean(0)]
        if tt % 10 == 0:
            gc.collect()
            torch.cuda.empty_cache()
ids = test_dl.dataset.ids
preds = torch.cat(preds, dim=0).float()
del test_dl, nets
gc.collect()
torch.cuda.empty_cache()

In [ ]:
preddf = pd.DataFrame(preds.numpy(), columns = ['patient_overall'] + cfg.target)
preddf['StudyInstanceUID'] = ids
preddf = preddf.set_index('StudyInstanceUID')
preddf.head()

In [45]:
preddf.shape

(5161, 8)

In [40]:
preddfls.append(preddf)

In [ ]:
preddfall = ((preddfls[0] * 1)+ (preddfls[1] * 4)) / 5
preddfall.head()

In [ ]:
subdf = preddfall.reset_index().melt(id_vars=["StudyInstanceUID"], var_name="row_id", value_name="fractured")
subdf['row_id'] = subdf[['StudyInstanceUID', 'row_id']] .agg('_'.join, axis=1)
subdf = subdf.drop('StudyInstanceUID', 1)
subdf.head()

In [44]:
subdf.shape

(41288, 2)

In [ ]:
subdf.to_csv(OUTPUT_FILE, index=False)